# Search Documents with Generative AI 📃



## Import Libraries

In [1]:
import os

from autogen import AssistantAgent, UserProxyAgent, register_function
from autogen.cache import Cache
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

True

## Connect to existing Azure OpenAI 🤖 & Azure Search Instances 🔎

In [5]:
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential, "https://cognitiveservices.azure.com/.default"
)
llm_config = {
    "config_list": [
        {
            "model":  "gpt-4o-deployment", # replace with your model deployment name
            "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
            "api_type": "azure",
            "api_version": "2024-05-01-preview",
            "max_tokens": 1000,
            "azure_ad_token_provider": token_provider
        }
    ],
    "cache_seed": 42,
    "temperature": 0,
    "timeout": 120

}

search_client = SearchClient(
    endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    index_name=os.getenv("AZURE_AI_SEARCH_INDEX"),
    credential=credential
)

openai_client = AzureOpenAI(
    azure_ad_token_provider=token_provider,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version="2024-05-01-preview"
)

def search(query: str):
    print("Searching for:", query)
    embeddings = openai_client.embeddings.create(
        model="text-embedding-3-small-deployment",
        input=[query]
    )
    vector_embedding = embeddings.data[0].embedding
    results = search_client.search(
        search_text=query,
        vector_queries=[{
            "kind": "vector",
            "vector": vector_embedding,
            "fields": "text_vector", # replace with your vector field name in the index
            "k": 10
        }],
        query_type="semantic",
        semantic_configuration_name=os.getenv("AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG"),
        top=10
    )

    output = []
    for result in results:
        result.pop("text_vector")
        output.append(result)

    return output


cog_search = AssistantAgent(
    name="COGSearch",
    system_message="You are a helpful AI assistant that uses Azure Cognitive Search to find relevant information. "
    "For each user query, use the search function to find information, then summarize the results in a helpful way. "
    "Cite specific documents when providing information. Be concise but thorough."
    "Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

register_function(
    search,
    caller=cog_search,
    executor=user_proxy,
    name="search",
    description="A tool for searching the Cognitive Search index",
)


## Question #1

In [6]:
cache = Cache.disk()
user_question = "document related to mesos and marathon troubleshooting guides"
await user_proxy.a_initiate_chat(
        cog_search,
        message=user_question,
        cache=cache,
    )

User (to COGSearch):

document related to mesos and marathon troubleshooting guides

--------------------------------------------------------------------------------
COGSearch (to User):

***** Suggested tool call (call_rRYZIPwqw1a8CnOt6JpATafZ): search *****
Arguments: 
{"query":"mesos and marathon troubleshooting guides"}
***********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search...
Call ID: call_rRYZIPwqw1a8CnOt6JpATafZ
Input arguments: {'query': 'mesos and marathon troubleshooting guides'}
Searching for: mesos and marathon troubleshooting guides
User (to COGSearch):

***** Response from calling tool (call_rRYZIPwqw1a8CnOt6JpATafZ) *****
[{'header_2': 'Actions', 'parent_id': 'aHR0cHM6Ly92ZWFpNHNmaW5wZXNhLmJsb2IuY29yZS53aW5kb3dzLm5ldC9maWxldXBsb2Fkcy90ZXN0ZGF0YS9SdW5ib29rcy1ha2EtVFNHcy9NZXNvcy1hbmQtTWFyYXRob24vTWVzb3MtU2VydmljZXMtUmVzdGFydGluZy5tZDsz0', '

ChatResult(chat_id=None, chat_history=[{'content': 'document related to mesos and marathon troubleshooting guides', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_rRYZIPwqw1a8CnOt6JpATafZ', 'function': {'arguments': '{"query":"mesos and marathon troubleshooting guides"}', 'name': 'search'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': '[{\'header_2\': \'Actions\', \'parent_id\': \'aHR0cHM6Ly92ZWFpNHNmaW5wZXNhLmJsb2IuY29yZS53aW5kb3dzLm5ldC9maWxldXBsb2Fkcy90ZXN0ZGF0YS9SdW5ib29rcy1ha2EtVFNHcy9NZXNvcy1hbmQtTWFyYXRob24vTWVzb3MtU2VydmljZXMtUmVzdGFydGluZy5tZDsz0\', \'header_3\': \'\', \'chunk\': \'|What|How|\\r\\n|--|--|\\r\\n| if the ICM Health history has a lot of ups & downs, this is likely a metrics issue, but continue debugging | ex. showing likely missing metrics, barely fired with a 11min gap ![image.png](/.attachments/image-a21ac253-adc2-49f5-9004-48fd800a02e9.png)\\r\\n| Check on bootstrap to see what/if services are restarting | browse

## Question #2

In [7]:
user_question = "service certificate rotation and ADO pipeline"
await user_proxy.a_initiate_chat(
        cog_search,
        message=user_question,
        cache=cache,
    )

User (to COGSearch):

service certificate rotation and ADO pipeline

--------------------------------------------------------------------------------
COGSearch (to User):

***** Suggested tool call (call_fWdFXizVMMzYWboBbI4HWyAW): search *****
Arguments: 
{"query":"service certificate rotation and Azure DevOps pipeline"}
***********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search...
Call ID: call_fWdFXizVMMzYWboBbI4HWyAW
Input arguments: {'query': 'service certificate rotation and Azure DevOps pipeline'}
Searching for: service certificate rotation and Azure DevOps pipeline
User (to COGSearch):

***** Response from calling tool (call_fWdFXizVMMzYWboBbI4HWyAW) *****
[{'header_2': 'Actions', 'parent_id': 'aHR0cHM6Ly92ZWFpNHNmaW5wZXNhLmJsb2IuY29yZS53aW5kb3dzLm5ldC9maWxldXBsb2Fkcy90ZXN0ZGF0YS9SdW5ib29rcy1ha2EtVFNHcy9TZWNyZXRzLVJvdGF0aW9uL1NlY3JldC1yb3RhdGlvbi1mb3

ChatResult(chat_id=None, chat_history=[{'content': 'service certificate rotation and ADO pipeline', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_fWdFXizVMMzYWboBbI4HWyAW', 'function': {'arguments': '{"query":"service certificate rotation and Azure DevOps pipeline"}', 'name': 'search'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': '[{\'header_2\': \'Actions\', \'parent_id\': \'aHR0cHM6Ly92ZWFpNHNmaW5wZXNhLmJsb2IuY29yZS53aW5kb3dzLm5ldC9maWxldXBsb2Fkcy90ZXN0ZGF0YS9SdW5ib29rcy1ha2EtVFNHcy9TZWNyZXRzLVJvdGF0aW9uL1NlY3JldC1yb3RhdGlvbi1mb3Itc2VydmljZS5tZDsy0\', \'header_3\': \'\', \'chunk\': \'- Identify what type of secret is expiring e.g. CERT, SAS TOKEN, or else\\r\\n- Identify the impacted service team.\\r\\n- Collaborate with the service team to rotate the secret. follow [Compute.wiki Certificates](https://dev.azure.com/yammer/Compute/_wiki/wikis/Compute.wiki/1062/Certificates)\\r\\n- Upload secret to Vault\\r\\n- Deploy service if needed 